In [1]:
import mne
import numpy as np
import pandas as pd
from model_functions import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from Classification_models_LOSO_2Class import *

In [2]:
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold,LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

Now, we get the connectivity matrix data for :
1) P3,P4,O1,O2 nodes for delta band with n=10 epochs
2) C3,C4,F3,F4 nodes for theta band with n=10 epochs
3) C3,C4,F3,F4 nodes for delta band with n=10 epochs
4) C3,C4,T3,T4 nodes for theta band with n=10 epochs
5) C3,C4,P3,P4 nodes for theta band with n=10 epochs

In [3]:
con_mats= np.load("connectivity_4nodes_delta.npy",allow_pickle = True)
con_mats_CF_theta= np.load("theta_C3C4F3F4_connectivity.npy",allow_pickle = True)
con_mats_CF_delta= np.load("C3C3F3F4_delta_connectivity.npy",allow_pickle = True)
#I made a typo in the above npy file. It should be C3C4F3F4
con_mats_CT_theta= np.load("C3C4T3T4_theta_connectivity.npy",allow_pickle = True)
con_mats_CP_theta= np.load("C3C4P3P4_theta_connectivity.npy",allow_pickle = True)

Next, we'll form an array regarding the connectivity datas from the symmetric nodes.

In [4]:
def con_arr(i):
    PO_delta_arr = [con_mats[i][1][0], con_mats[i][2][0],con_mats[i][2][1],con_mats[i][3][0],con_mats[i][3][1],con_mats[i][3][2]] 
    CF_theta_arr = [con_mats_CF_theta[i][0][1][0], con_mats_CF_theta[i][0][2][0],con_mats_CF_theta[i][0][2][1],con_mats_CF_theta[i][0][3][0],con_mats_CF_theta[i][0][3][1],con_mats_CF_theta[i][0][3][2]] 
    CF_delta_arr = [con_mats_CF_delta[i][0][1][0], con_mats_CF_delta[i][0][2][0],con_mats_CF_delta[i][0][2][1],con_mats_CF_delta[i][0][3][0],con_mats_CF_delta[i][0][3][1],con_mats_CF_delta[i][0][3][2]] 
    CT_theta_arr = [con_mats_CT_theta[i][0][1][0], con_mats_CT_theta[i][0][2][0],con_mats_CT_theta[i][0][2][1],con_mats_CT_theta[i][0][3][0],con_mats_CT_theta[i][0][3][1],con_mats_CT_theta[i][0][3][2]] 
    CP_theta_arr = [con_mats_CP_theta[i][0][1][0], con_mats_CP_theta[i][0][2][0],con_mats_CP_theta[i][0][2][1],con_mats_CP_theta[i][0][3][0],con_mats_CP_theta[i][0][3][1],con_mats_CP_theta[i][0][3][2]]
    
    return PO_delta_arr, CF_theta_arr, CF_delta_arr, CT_theta_arr, CP_theta_arr

Next, we'll remove test set (5 Alzheimer's and 3 healthy subjects) and put their data in a DataFrame.

In [5]:
test_list= [4,6,7,20,33,49,53,63]

targets=[]
targets=[1 for i in range(36)]
targets.extend([0 for i in range(29)])

test_data= {"P4P3": [con_arr(i)[0][0] for i in test_list],
        "O1P3": [con_arr(i)[0][1] for i in test_list],
        "O1P4": [con_arr(i)[0][2] for i in test_list],
        "O2P3": [con_arr(i)[0][3] for i in test_list],
        "O2P4": [con_arr(i)[0][4] for i in test_list],
        "O2O1": [con_arr(i)[0][5] for i in test_list],
        
        "F4F3_theta": [con_arr(i)[1][0] for i in test_list],
        "C3F3_theta": [con_arr(i)[1][1] for i in test_list],
        "C3F4_theta": [con_arr(i)[1][2] for i in test_list],
        "C4F3_theta": [con_arr(i)[1][3] for i in test_list],
        "C4F4_theta": [con_arr(i)[1][4] for i in test_list],
        "C4C3_theta0": [con_arr(i)[1][5] for i in test_list],
        
        "F4F3_delta": [con_arr(i)[2][0] for i in test_list],
        "C3F3_delta": [con_arr(i)[2][1] for i in test_list],
        "C3F4_delta": [con_arr(i)[2][2] for i in test_list],
        "C4F3_delta": [con_arr(i)[2][3] for i in test_list],
        "C4F4_delta": [con_arr(i)[2][4] for i in test_list],
        "C4C3_delta": [con_arr(i)[2][5] for i in test_list],
        
        "T4T3_theta": [con_arr(i)[3][0] for i in test_list],
        "C3T3_theta": [con_arr(i)[3][1] for i in test_list],
        "C3T4_theta": [con_arr(i)[3][2] for i in test_list],
        "C4T3_theta": [con_arr(i)[3][3] for i in test_list],
        "C4T4_theta": [con_arr(i)[3][4] for i in test_list],
        "C4C3_theta1": [con_arr(i)[3][5] for i in test_list],
        
        "P4P3_theta": [con_arr(i)[4][0] for i in test_list],
        "C3P3_theta": [con_arr(i)[4][1] for i in test_list],
        "C3P4_theta": [con_arr(i)[4][2] for i in test_list],
        "C4P3_theta": [con_arr(i)[4][3] for i in test_list],
        "C4P4_theta": [con_arr(i)[4][4] for i in test_list],
        "C4C3_theta2": [con_arr(i)[4][5] for i in test_list],
        
        "Category":[targets[i] for i in test_list]
       }

In [6]:
my_test_df = pd.DataFrame(test_data)
my_test_df

,P4P3,O1P3,O1P4,O2P3,O2P4,O2O1,F4F3_theta,C3F3_theta,C3F4_theta,C4F3_theta,...,C4T3_theta,C4T4_theta,C4C3_theta1,P4P3_theta,C3P3_theta,C3P4_theta,C4P3_theta,C4P4_theta,C4C3_theta2,Category
0,0.772771,0.887677,0.785512,0.765683,0.845178,0.878390,0.837810,0.822111,0.753706,0.736878,...,0.685136,0.816485,0.696001,0.716960,0.814071,0.678475,0.687119,0.854531,0.780376,1
1,0.654467,0.912244,0.715622,0.743344,0.870926,0.829239,0.772350,0.779367,0.654680,0.588947,...,0.607852,0.738132,0.702734,0.572938,0.835573,0.556740,0.597742,0.814132,0.635498,1
2,0.649615,0.863933,0.743501,0.730586,0.850091,0.885674,0.682731,0.780380,0.565793,0.545300,...,0.555428,0.763650,0.709009,0.547242,0.723655,0.602015,0.555474,0.795883,0.782493,1
3,0.728172,0.825603,0.729901,0.738757,0.865315,0.853545,0.787582,0.832154,0.727096,0.723429,...,0.658060,0.779146,0.656754,0.746135,0.837895,0.683551,0.712011,0.857232,0.775777,1
4,0.596776,0.859832,0.666745,0.715443,0.836781,0.833949,0.778868,0.889055,0.733139,0.695717,...,0.641710,0.806904,0.675673,0.679945,0.860345,0.678085,0.657916,0.845574,0.733133,1
5,0.726735,0.859820,0.721827,0.730662,0.830195,0.836218,0.788242,0.775022,0.629512,0.678210,...,0.537094,0.726794,0.612954,0.652871,0.713457,0.550103,0.581414,0.700903,0.770426,0
6,0.480559,0.545285,0.545577,0.560528,0.640305,0.471714,0.732385,0.540710,0.335434,0.354633,...,0.299443,0.681961,0.497177,0.290147,0.635464,0.301016,0.309869,0.557280,0.480424,0
7,0.404359,0.695890,0.565609,0.459311,0.748539,0.725173,0.785046,0.386158,0.294661,0.331795,...,0.293924,0.527590,0.486851,0.340366,0.558214,0.304936,0.294133,0.574622,0.369678,0


In [7]:
my_test_list= [4,6,7,20,33,49,53,63]
my_train_list= [0,1,2,3,5,7,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,50,51,52,54,55,56,57,58,59,60,61,62,64]

Next, we will form our DataFrame.

In [8]:
#For each of 36 AD and 29 CN subjects, we have mean of persistance arrays.
targets=[]
targets=[1 for i in range(33)]
targets.extend([0 for i in range(26)])

data = {"P4P3": [con_arr(i)[0][0] for i in my_train_list],
        "O1P3": [con_arr(i)[0][1] for i in my_train_list],
        "O1P4": [con_arr(i)[0][2] for i in my_train_list],
        "O2P3": [con_arr(i)[0][3] for i in my_train_list],
        "O2P4": [con_arr(i)[0][4] for i in my_train_list],
        "O2O1": [con_arr(i)[0][5] for i in my_train_list],
        
        "F4F3_theta": [con_arr(i)[1][0] for i in my_train_list],
        "C3F3_theta": [con_arr(i)[1][1] for i in my_train_list],
        "C3F4_theta": [con_arr(i)[1][2] for i in my_train_list],
        "C4F3_theta": [con_arr(i)[1][3] for i in my_train_list],
        "C4F4_theta": [con_arr(i)[1][4] for i in my_train_list],
        "C4C3_theta0": [con_arr(i)[1][5] for i in my_train_list],
        
        "F4F3_delta": [con_arr(i)[2][0] for i in my_train_list],
        "C3F3_delta": [con_arr(i)[2][1] for i in my_train_list],
        "C3F4_delta": [con_arr(i)[2][2] for i in my_train_list],
        "C4F3_delta": [con_arr(i)[2][3] for i in my_train_list],
        "C4F4_delta": [con_arr(i)[2][4] for i in my_train_list],
        "C4C3_delta": [con_arr(i)[2][5] for i in my_train_list],
        
        "T4T3_theta": [con_arr(i)[3][0] for i in my_train_list],
        "C3T3_theta": [con_arr(i)[3][1] for i in my_train_list],
        "C3T4_theta": [con_arr(i)[3][2] for i in my_train_list],
        "C4T3_theta": [con_arr(i)[3][3] for i in my_train_list],
        "C4T4_theta": [con_arr(i)[3][4] for i in my_train_list],
        "C4C3_theta1": [con_arr(i)[3][5] for i in my_train_list],
        
        "P4P3_theta": [con_arr(i)[4][0] for i in my_train_list],
        "C3P3_theta": [con_arr(i)[4][1] for i in my_train_list],
        "C3P4_theta": [con_arr(i)[4][2] for i in my_train_list],
        "C4P3_theta": [con_arr(i)[4][3] for i in my_train_list],
        "C4P4_theta": [con_arr(i)[4][4] for i in my_train_list],
        "C4C3_theta2": [con_arr(i)[4][5] for i in my_train_list],
        
        "Category":[targets[i] for i in range(0,57)]
       }


In [9]:
my_df= pd.DataFrame(data)
len(my_df)

57

In [10]:
def train_prep(my_df, i):
    X = my_df.copy()
    y = X.Category
    
    # The following two arrays will store the target and features for the train set. We will append to it
    # as we go through various subjects.
    X_train = X.drop(index=i)
    y_train = X_train.Category
        
    # The following two arrays will store target and features for the test set. 
    X_test = pd.DataFrame(X.iloc[[i]])
    y_test = pd.DataFrame(y.iloc[[i]])    
    
    return X_train, y_train, X_test, y_test

In [15]:
def svm_cross(my_df, degree):
    confusion_matrices_train = []
    confusion_matrices_test = []
    labels = [0,1]
    
    for i in range(len(my_df)):
        X_train, y_train, X_test, y_test = train_prep(my_df, i)
        
        ## Make the model object
        svc = SVC(kernel='poly', degree=degree, C=10)

        ## Fit the model object
        svc.fit(X_train[['P4P3', 'O1P3', 'O1P4', 'O2P3', 'O2P4', 'O2O1', 'F4F3_theta',
       'C3F3_theta', 'C3F4_theta', 'C4F3_theta', 'C4F4_theta', 'C4C3_theta0',
       'F4F3_delta', 'C3F3_delta', 'C3F4_delta', 'C4F3_delta', 'C4F4_delta',
       'C4C3_delta', 'T4T3_theta', 'C3T3_theta', 'C3T4_theta', 'C4T3_theta',
       'C4T4_theta', 'C4C3_theta1', 'P4P3_theta', 'C3P3_theta', 'C3P4_theta',
       'C4P3_theta', 'C4P4_theta', 'C4C3_theta2']],y_train)

        ## get the predictions
        y_train_pred = svc.predict(X_train[['P4P3', 'O1P3', 'O1P4', 'O2P3', 'O2P4', 'O2O1', 'F4F3_theta',
       'C3F3_theta', 'C3F4_theta', 'C4F3_theta', 'C4F4_theta', 'C4C3_theta0',
       'F4F3_delta', 'C3F3_delta', 'C3F4_delta', 'C4F3_delta', 'C4F4_delta',
       'C4C3_delta', 'T4T3_theta', 'C3T3_theta', 'C3T4_theta', 'C4T3_theta',
       'C4T4_theta', 'C4C3_theta1', 'P4P3_theta', 'C3P3_theta', 'C3P4_theta',
       'C4P3_theta', 'C4P4_theta', 'C4C3_theta2']].values)
        y_test_pred = svc.predict(X_test[['P4P3', 'O1P3', 'O1P4', 'O2P3', 'O2P4', 'O2O1', 'F4F3_theta',
       'C3F3_theta', 'C3F4_theta', 'C4F3_theta', 'C4F4_theta', 'C4C3_theta0',
       'F4F3_delta', 'C3F3_delta', 'C3F4_delta', 'C4F3_delta', 'C4F4_delta',
       'C4C3_delta', 'T4T3_theta', 'C3T3_theta', 'C3T4_theta', 'C4T3_theta',
       'C4T4_theta', 'C4C3_theta1', 'P4P3_theta', 'C3P3_theta', 'C3P4_theta',
       'C4P3_theta', 'C4P4_theta', 'C4C3_theta2']].values)
        
        labels=[0,1]
        confusion_matrices_train += [confusion_matrix(y_train, y_train_pred,labels=labels)]
        confusion_matrices_test += [confusion_matrix((y_test.Category).to_numpy(),y_test_pred, labels=labels)]
    
    confusion_matrices_train = np.array(confusion_matrices_train)
    confusion_matrices_test = np.array(confusion_matrices_test)
    total_confusion_train = np.sum(confusion_matrices_train, axis= 0)
    total_confusion_test = np.sum(confusion_matrices_test, axis= 0)
    
    train_metrics_dict = {'acc':accuracy(total_confusion_train), 'sens':sensitivity(total_confusion_train), 
                            'spec':specificity(total_confusion_train), 'f1':f1(total_confusion_train)}
    test_metrics_dict = {'acc':accuracy(total_confusion_test), 'sens':sensitivity(total_confusion_test), 
                            'spec':specificity(total_confusion_test), 'f1':f1(total_confusion_test)}
    return train_metrics_dict, test_metrics_dict

In [16]:
svm_cross(my_df,3)

({'acc': 1.0, 'sens': 1.0, 'spec': 1.0, 'f1': 1.0},
 {'acc': 0.8421052631578947,
  'sens': 0.8484848484848485,
  'spec': 0.8333333333333334,
  'f1': 0.8615384615384615})

In [18]:
svm_cross(my_test_df,3)

({'acc': 1.0, 'sens': 1.0, 'spec': 1.0, 'f1': 1.0},
 {'acc': 0.875,
  'sens': 1.0,
  'spec': 0.6666666666666666,
  'f1': 0.9090909090909091})